In [1]:
from pathlib import Path 
import os, dotenv, yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

dotenv.load_dotenv()
os.chdir(Path(config["pythonpath"]).expanduser())

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

In [3]:
VARIATION = "nocomments"
VERSION = config["version"]

source_code_df = pd.read_json(f"data/clean_datasets/{VARIATION}_{VERSION}.jsonl", lines=True)
source_code_df['filetype'] = source_code_df['filetype'].apply(
    lambda x: 'cryptol' if x == 'cry' else 'saw' if x == 'saw' else 'text'
    )

mcc_df = pd.read_json(f"data/clean_datasets/mcc_stats_{VARIATION}_{VERSION}.jsonl", lines=True)

add_columns = mcc_df.columns.difference(source_code_df.columns).tolist()
print(add_columns)
source_code_df = source_code_df.merge(
    mcc_df[["filename", *add_columns]],
    on="filename",
    how="left",
    validate="one_to_one"
)
source_code_df.head()


['avg_mcc', 'avg_mcc_difficulty_bucket', 'imports', 'imports_count', 'json_path', 'max_mcc', 'num_declarations', 'num_declarations_with_mcc', 'num_definitions', 'num_types', 'split', 'total_mcc']


,filename,filetype,content,variant,n_imports_original,n_imports_final,avg_mcc,avg_mcc_difficulty_bucket,imports,imports_count,json_path,max_mcc,num_declarations,num_declarations_with_mcc,num_definitions,num_types,split,total_mcc
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,cryptol,module AES where\n\nimport `Common::AES\n\ntyp...,without_comments,NaN,NaN,1.000000,1,[`Common::AES],1,/Users/josh/Automated_Reasoning_for_Cryptograp...,1,4,4,5,1,train,4
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,cryptol,module AES128 where\n\nimport `Common::AES\nim...,without_comments,NaN,NaN,1.000000,1,"[`Common::AES, `Common::AES as AES, AES as AES]",3,/Users/josh/Automated_Reasoning_for_Cryptograp...,1,6,6,9,3,test,6
2,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,cryptol,module AES256 where\n\nimport `Common::AES\nim...,without_comments,NaN,NaN,1.000000,1,"[`Common::AES, `Common::AES as AES, AES as AES]",3,/Users/josh/Automated_Reasoning_for_Cryptograp...,1,6,6,9,3,train,6
3,AES-GCM-SIV-proof/proof/cryptol-specs/TBox.cry,cryptol,type Nb = 4\ntype State = [4][Nb]...,without_comments,NaN,NaN,1.352941,4,[],0,/Users/josh/Automated_Reasoning_for_Cryptograp...,3,17,17,21,4,train,23
4,AES-GCM-SIV-proof/proof/cryptol-specs/Common/A...,cryptol,module Common::AES where\n\nparameter\n type ...,without_comments,NaN,NaN,1.358491,4,[],0,/Users/josh/Automated_Reasoning_for_Cryptograp...,3,53,53,61,8,train,72


In [4]:
from src.data_s.util import build_masked_examples_df

masked_df = build_masked_examples_df(source_code_df=source_code_df)

#masked_df = masked_df.merge(
#    mcc_df[["filename", "num_declarations", "total_mcc"]],
#    on="filename",
#    how="left"
#)

masked_df.head()

DEBUG name hits: [(3, '"fnv1a ws = fnv1a\' (pad ws)"'), (8, '"fnv1a\' : {chunks} (fin chunks) => [chunks][8] -> [64]"'), (9, '"fnv1a\' msg = Ss ! 0"')]
DEBUG lstrip startswith hits: [(8, '"fnv1a\' : {chunks} (fin chunks) => [chunks][8] -> [64]"'), (9, '"fnv1a\' msg = Ss ! 0"')]
Error processing aws-lc-verification/cryptol-specs/Primitive/Keyless/Hash/FNV.cry: Could not find definition span for 'fnv1a''
DEBUG name hits: [(44, '"  new  = G\' r i (vec @@ abcd) ms"'), (46, '"G\' : [4] -> [4] -> [4][64] -> Block -> [4][64]"'), (47, '"G\' r i vals ms = [a2, b2, c2, d2]"')]
DEBUG lstrip startswith hits: [(46, '"G\' : [4] -> [4] -> [4][64] -> Block -> [4][64]"'), (47, '"G\' r i vals ms = [a2, b2, c2, d2]"')]
Error processing aws-lc-verification/cryptol-specs/Primitive/Keyless/Hash/Blake2b.cry: Could not find definition span for 'G''
DEBUG name hits: [(34, "'p384_field_mul(x,y)      = p384_field_mod(p384_safe_product(x, y))'"), (47, "'p384_field_mod : [768] -> [384]'"), (48, "'p384_field_mod(a)

,filename,filetype,content,variant,n_imports_original,n_imports_final,avg_mcc,avg_mcc_difficulty_bucket,imports,imports_count,...,num_declarations_with_mcc,num_definitions,num_types,split,total_mcc,def_name,def_params,masked_source,target_definition,hole_name
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,cryptol,module AES where\n\nimport `Common::AES\n\ntyp...,without_comments,NaN,NaN,1.000000,1,[`Common::AES],1,...,4,5,1,train,4,encrypt,[],module AES where\n\nimport `Common::AES\n\ntyp...,encrypt = aesEncrypt`{Mode = m}\n,/* Finish this definition. */
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,cryptol,module AES128 where\n\nimport `Common::AES\nim...,without_comments,NaN,NaN,1.000000,1,"[`Common::AES, `Common::AES as AES, AES as AES]",3,...,6,9,3,test,6,aes_final_round128,[],module AES128 where\n\nimport `Common::AES\nim...,aes_final_round128 = AES::aes_final_round`{0}\n,/* Finish this definition. */
2,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,cryptol,module AES256 where\n\nimport `Common::AES\nim...,without_comments,NaN,NaN,1.000000,1,"[`Common::AES, `Common::AES as AES, AES as AES]",3,...,6,9,3,train,6,aes_final_round256,[],module AES256 where\n\nimport `Common::AES\nim...,aes_final_round256 = AES::aes_final_round`{2}\n,/* Finish this definition. */
3,AES-GCM-SIV-proof/proof/cryptol-specs/TBox.cry,cryptol,type Nb = 4\ntype State = [4][Nb]...,without_comments,NaN,NaN,1.352941,4,[],0,...,17,21,4,train,23,gf28Pow,"[k, n]",type Nb = 4\ntype State = [4][Nb]...,gf28Pow n k = pow k\n where sq x = gf28Mul...,/* Finish this definition. */
4,AES-GCM-SIV-proof/proof/cryptol-specs/Common/A...,cryptol,module Common::AES where\n\nparameter\n type ...,without_comments,NaN,NaN,1.358491,4,[],0,...,53,61,8,train,72,gf28Pow,"[k, n]",module Common::AES where\n\nparameter\n type ...,"gf28Pow (n, k) = pow k\n where sq x = gf28...",/* Finish this definition. */


In [5]:
masked_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   filename                   730 non-null    object 
 1   filetype                   730 non-null    object 
 2   content                    730 non-null    object 
 3   variant                    280 non-null    object 
 4   n_imports_original         450 non-null    float64
 5   n_imports_final            450 non-null    float64
 6   avg_mcc                    730 non-null    float64
 7   avg_mcc_difficulty_bucket  730 non-null    int64  
 8   imports                    730 non-null    object 
 9   imports_count              730 non-null    int64  
 10  json_path                  730 non-null    object 
 11  max_mcc                    730 non-null    int64  
 12  num_declarations           730 non-null    int64  
 13  num_declarations_with_mcc  730 non-null    int64  

In [6]:
from src.agent.generate import ModelConfig, TemplateBundle, iter_call_masked_instruction_pydantic_ai, build_prompt_call_pydantic_ai


file_cache_path = Path(f"cache/alpaca_instruct_cache/SFT_{VARIATION}_Masked_Instruction_{VERSION}.jsonl")
file_cache_path.parent.mkdir(parents=True, exist_ok=True)  # create dirs if missing

#test_df = source_code_df.sample(10, random_state=42).reset_index(drop=True)

modelConfig = ModelConfig(
    model="openai:gpt-5.1",
    retries=2
)

template = TemplateBundle(
    spec_system="system_spec_masked_instruction.j2",
    spec_user="user_spec_masked_instruction.j2"
)

result = iter_call_masked_instruction_pydantic_ai(
    masked_df,
    model_cfg=modelConfig,
    spec_templates=template,
    input_mode="none",
    file_cache_path=file_cache_path
)
result.head()

Processing row 0 / 730: AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry
Processing row 10 / 730: aws-lc-verification/cryptol-specs/Common/mul_java.cry
Processing row 20 / 730: aws-lc-verification/cryptol-specs/Primitive/Symmetric/Cipher/Block/AES_parameterized.cry
Processing row 30 / 730: aws-lc-verification/cryptol-specs/Primitive/Symmetric/Cipher/Block/Modes/TDES_CFB.cry
Processing row 40 / 730: aws-lc-verification/cryptol-specs/Primitive/Symmetric/Cipher/Block/AES/TBox.cry
Processing row 50 / 730: aws-lc-verification/cryptol-specs/McEliece_KEM/low-level/types.cry
Processing row 60 / 730: aws-lc-verification/cryptol-specs/McEliece_KEM/high-level/util.cry
Processing row 70 / 730: aws-lc-verification/cryptol-specs-aes-gcm/Common/LUP.cry
Processing row 80 / 730: aws-lc-verification/cryptol-specs-aes-gcm/Primitive/Keyless/Hash/utils.cry
Processing row 90 / 730: aws-lc-verification/cryptol-specs-aes-gcm/McEliece_KEM/high-level/Key_Generation.cry
Processing row 100 / 730: BLST-Verification/s

,filename,filetype,content,variant,n_imports_original,n_imports_final,avg_mcc,avg_mcc_difficulty_bucket,imports,imports_count,...,split,total_mcc,def_name,def_params,masked_source,target_definition,hole_name,instruction,input,output
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,cryptol,module AES where\n\nimport `Common::AES\n\ntyp...,without_comments,NaN,NaN,1.000000,1,[`Common::AES],1,...,train,4,encrypt,[],module AES where\n\nimport `Common::AES\n\ntyp...,encrypt = aesEncrypt`{Mode = m}\n,/* Finish this definition. */,"In this Cryptol module AES, define and briefly...",,
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,cryptol,module AES128 where\n\nimport `Common::AES\nim...,without_comments,NaN,NaN,1.000000,1,"[`Common::AES, `Common::AES as AES, AES as AES]",3,...,test,6,aes_final_round128,[],module AES128 where\n\nimport `Common::AES\nim...,aes_final_round128 = AES::aes_final_round`{0}\n,/* Finish this definition. */,"In the AES128 Cryptol module, define aes_final...",,
2,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,cryptol,module AES256 where\n\nimport `Common::AES\nim...,without_comments,NaN,NaN,1.000000,1,"[`Common::AES, `Common::AES as AES, AES as AES]",3,...,train,6,aes_final_round256,[],module AES256 where\n\nimport `Common::AES\nim...,aes_final_round256 = AES::aes_final_round`{2}\n,/* Finish this definition. */,"In the AES256 Cryptol module, define and brief...",,
3,AES-GCM-SIV-proof/proof/cryptol-specs/TBox.cry,cryptol,type Nb = 4\ntype State = [4][Nb]...,without_comments,NaN,NaN,1.352941,4,[],0,...,train,23,gf28Pow,"[k, n]",type Nb = 4\ntype State = [4][Nb]...,gf28Pow n k = pow k\n where sq x = gf28Mul...,/* Finish this definition. */,"In this Cryptol AES GF(2^8) specification, def...",,
4,AES-GCM-SIV-proof/proof/cryptol-specs/Common/A...,cryptol,module Common::AES where\n\nparameter\n type ...,without_comments,NaN,NaN,1.358491,4,[],0,...,train,72,gf28Pow,"[k, n]",module Common::AES where\n\nparameter\n type ...,"gf28Pow (n, k) = pow k\n where sq x = gf28...",/* Finish this definition. */,"In the Common::AES Cryptol module, define gf28...",,


In [7]:

import textwrap

WRAP = 80

result_sample = result.sample(20, random_state=42).reset_index(drop=True)

for idx, row in result_sample.iterrows():
    print(f"=== Example {idx}: {row['filename']} ===")
    instr = str(row["instruction"] or "")
    wrapped_instr = "\n\n".join(
        textwrap.fill(p, width=WRAP, break_long_words=False, break_on_hyphens=False)
        for p in instr.splitlines()
    )
    print(f"Instruction:\n{wrapped_instr}\n")
    print(f"Source Code:\n{'=' * WRAP}\n{row['masked_source']}\n{'=' * WRAP}")



=== Example 0: cryptol_slices/cryptol/docs/ProgrammingCryptol/enigma/Enigma/017_reflector.cry ===
Instruction:
In this Cryptol code, define and briefly explain the reflector constant.

Source Code:
type Permutation = String 26
type Reflector = Permutation

reflector : Reflector
reflector = /* Finish this definition. */

=== Example 1: cryptol/examples/MiniLock/prim/bv.cry ===
Instruction:
In the bv module, define and briefly explain adc for n-bit inputs x and y,
returning n-bit sum and carry-out Bit, using safe_add.

Source Code:
module bv where

uext : {a, b} (fin b,fin a) => [a] -> [a+b]
uext(x) = zero # x

iext : {a, b} (fin b,fin a) => [a] -> [a+b]
iext(x) = (if x@0 then ~zero else zero) # x

safe_add : {n} (fin n) => ([n],[n]) -> [n+1]
safe_add(x,y) = uext(x) + uext(y)

safe_sub : {n} (fin n) => ([n],[n]) -> [n+1]
safe_sub(x,y) = uext(x) - uext(y)

is_normal : {a} (fin a) => [a] -> [a] -> Bit
is_normal p x = (x < p)

safe_product : {a} (fin a) => ([a],[a]) -> [2*a]
safe_product(x,

In [8]:
masked_df.head()

,filename,filetype,content,variant,n_imports_original,n_imports_final,avg_mcc,avg_mcc_difficulty_bucket,imports,imports_count,...,num_declarations_with_mcc,num_definitions,num_types,split,total_mcc,def_name,def_params,masked_source,target_definition,hole_name
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,cryptol,module AES where\n\nimport `Common::AES\n\ntyp...,without_comments,NaN,NaN,1.000000,1,[`Common::AES],1,...,4,5,1,train,4,encrypt,[],module AES where\n\nimport `Common::AES\n\ntyp...,encrypt = aesEncrypt`{Mode = m}\n,/* Finish this definition. */
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,cryptol,module AES128 where\n\nimport `Common::AES\nim...,without_comments,NaN,NaN,1.000000,1,"[`Common::AES, `Common::AES as AES, AES as AES]",3,...,6,9,3,test,6,aes_final_round128,[],module AES128 where\n\nimport `Common::AES\nim...,aes_final_round128 = AES::aes_final_round`{0}\n,/* Finish this definition. */
2,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,cryptol,module AES256 where\n\nimport `Common::AES\nim...,without_comments,NaN,NaN,1.000000,1,"[`Common::AES, `Common::AES as AES, AES as AES]",3,...,6,9,3,train,6,aes_final_round256,[],module AES256 where\n\nimport `Common::AES\nim...,aes_final_round256 = AES::aes_final_round`{2}\n,/* Finish this definition. */
3,AES-GCM-SIV-proof/proof/cryptol-specs/TBox.cry,cryptol,type Nb = 4\ntype State = [4][Nb]...,without_comments,NaN,NaN,1.352941,4,[],0,...,17,21,4,train,23,gf28Pow,"[k, n]",type Nb = 4\ntype State = [4][Nb]...,gf28Pow n k = pow k\n where sq x = gf28Mul...,/* Finish this definition. */
4,AES-GCM-SIV-proof/proof/cryptol-specs/Common/A...,cryptol,module Common::AES where\n\nparameter\n type ...,without_comments,NaN,NaN,1.358491,4,[],0,...,53,61,8,train,72,gf28Pow,"[k, n]",module Common::AES where\n\nparameter\n type ...,"gf28Pow (n, k) = pow k\n where sq x = gf28...",/* Finish this definition. */


In [9]:
masked_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   filename                   730 non-null    object 
 1   filetype                   730 non-null    object 
 2   content                    730 non-null    object 
 3   variant                    280 non-null    object 
 4   n_imports_original         450 non-null    float64
 5   n_imports_final            450 non-null    float64
 6   avg_mcc                    730 non-null    float64
 7   avg_mcc_difficulty_bucket  730 non-null    int64  
 8   imports                    730 non-null    object 
 9   imports_count              730 non-null    int64  
 10  json_path                  730 non-null    object 
 11  max_mcc                    730 non-null    int64  
 12  num_declarations           730 non-null    int64  
 13  num_declarations_with_mcc  730 non-null    int64  

In [10]:
from src.agent.generate import iter_call_masked_define_explain_pydantic_ai

templates = TemplateBundle(
    spec_system="system_spec_masked_explain.j2",
    spec_user="user_spec_masked_explain.j2",
)
file_cache = "cache/alpaca_instruct_cache/TESTSFT_nocomments_Masked_Instruction_v1-0.jsonl"
explained_df = iter_call_masked_define_explain_pydantic_ai(
    result,   # must contain instruction + masked_source + target_definition etc
    input_mode="full",
    file_cache_path=file_cache,
    model_cfg=ModelConfig(model="openai:gpt-5.1", retries=2),
    spec_templates=templates,
)


Explain stage 0 / 730: AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry::encrypt
Explain stage 10 / 730: aws-lc-verification/cryptol-specs/Common/mul_java.cry::mul_java
Explain stage 20 / 730: aws-lc-verification/cryptol-specs/Primitive/Symmetric/Cipher/Block/AES_parameterized.cry::decrypt
Explain stage 30 / 730: aws-lc-verification/cryptol-specs/Primitive/Symmetric/Cipher/Block/Modes/TDES_CFB.cry::cfbTDesEnc
Explain stage 40 / 730: aws-lc-verification/cryptol-specs/Primitive/Symmetric/Cipher/Block/AES/TBox.cry::mkTBox
Explain stage 50 / 730: aws-lc-verification/cryptol-specs/McEliece_KEM/low-level/types.cry::COND_BYTES
Explain stage 60 / 730: aws-lc-verification/cryptol-specs/McEliece_KEM/high-level/util.cry::bitrev
Explain stage 70 / 730: aws-lc-verification/cryptol-specs-aes-gcm/Common/LUP.cry::LUPSolve
Explain stage 80 / 730: aws-lc-verification/cryptol-specs-aes-gcm/Primitive/Keyless/Hash/utils.cry::fromBytes
Explain stage 90 / 730: aws-lc-verification/cryptol-specs-aes-gcm/McEliece_

In [11]:
explained_df.head()

,filename,filetype,content,variant,n_imports_original,n_imports_final,avg_mcc,avg_mcc_difficulty_bucket,imports,imports_count,...,total_mcc,def_name,def_params,masked_source,target_definition,hole_name,instruction,input,output,explanation
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,cryptol,module AES where\n\nimport `Common::AES\n\ntyp...,without_comments,NaN,NaN,1.000000,1,[`Common::AES],1,...,4,encrypt,[],module AES where\n\nimport `Common::AES\n\ntyp...,encrypt = aesEncrypt`{Mode = m}\n,/* Finish this definition. */,"In this Cryptol module AES, define and briefly...",,,encrypt takes a valid-size AES key and a 128-b...
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,cryptol,module AES128 where\n\nimport `Common::AES\nim...,without_comments,NaN,NaN,1.000000,1,"[`Common::AES, `Common::AES as AES, AES as AES]",3,...,6,aes_final_round128,[],module AES128 where\n\nimport `Common::AES\nim...,aes_final_round128 = AES::aes_final_round`{0}\n,/* Finish this definition. */,"In the AES128 Cryptol module, define aes_final...",,,aes_final_round128 is the 128-bit AES final-ro...
2,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,cryptol,module AES256 where\n\nimport `Common::AES\nim...,without_comments,NaN,NaN,1.000000,1,"[`Common::AES, `Common::AES as AES, AES as AES]",3,...,6,aes_final_round256,[],module AES256 where\n\nimport `Common::AES\nim...,aes_final_round256 = AES::aes_final_round`{2}\n,/* Finish this definition. */,"In the AES256 Cryptol module, define and brief...",,,aes_final_round256 is the AES-256 final-round ...
3,AES-GCM-SIV-proof/proof/cryptol-specs/TBox.cry,cryptol,type Nb = 4\ntype State = [4][Nb]...,without_comments,NaN,NaN,1.352941,4,[],0,...,23,gf28Pow,"[k, n]",type Nb = 4\ntype State = [4][Nb]...,gf28Pow n k = pow k\n where sq x = gf28Mul...,/* Finish this definition. */,"In this Cryptol AES GF(2^8) specification, def...",,,gf28Pow raises a GF(2^8) element to an 8-bit e...
4,AES-GCM-SIV-proof/proof/cryptol-specs/Common/A...,cryptol,module Common::AES where\n\nparameter\n type ...,without_comments,NaN,NaN,1.358491,4,[],0,...,72,gf28Pow,"[k, n]",module Common::AES where\n\nparameter\n type ...,"gf28Pow (n, k) = pow k\n where sq x = gf28...",/* Finish this definition. */,"In the Common::AES Cryptol module, define gf28...",,,gf28Pow computes exponentiation in the AES fin...


In [12]:
import textwrap

for idx, row in explained_df.sample(10, random_state=42).iterrows():
    print(f"=== Example {idx}: {row['filename']} ===")
    print(f"Task:\n{textwrap.fill(row['instruction'], width=100)}\n")
    print(f"```cryptol\n{row['masked_source']}\n```")
    print(f"Solution:\n```cryptol\n{row['target_definition']}\n```")
    print(f"Explanation:\n{textwrap.fill(row['explanation'], width=100)}\n")

=== Example 468: cryptol_slices/cryptol/docs/ProgrammingCryptol/enigma/Enigma/017_reflector.cry ===
Task:
In this Cryptol code, define and briefly explain the reflector constant.

```cryptol
type Permutation = String 26
type Reflector = Permutation

reflector : Reflector
reflector = /* Finish this definition. */

```
Solution:
```cryptol
reflector = "FEIPBATSCYVUWZQDOXHGLKMRJN"
```
Explanation:
A fixed 26-character wiring permutation modeling the Enigma machine’s reflector mapping.

=== Example 148: cryptol/examples/MiniLock/prim/bv.cry ===
Task:
In the bv module, define and briefly explain adc for n-bit inputs x and y, returning n-bit sum and
carry-out Bit, using safe_add.

```cryptol
module bv where

uext : {a, b} (fin b,fin a) => [a] -> [a+b]
uext(x) = zero # x

iext : {a, b} (fin b,fin a) => [a] -> [a+b]
iext(x) = (if x@0 then ~zero else zero) # x

safe_add : {n} (fin n) => ([n],[n]) -> [n+1]
safe_add(x,y) = uext(x) + uext(y)

safe_sub : {n} (fin n) => ([n],[n]) -> [n+1]
safe_sub(x

In [13]:
explained_df["scenario"] = "MASKED_EXPLAIN_CODE"
explained_df["explanation"] = explained_df["explanation"].fillna("").apply(lambda s: textwrap.fill(str(s), width=80))
explained_df["instruction"] = explained_df["instruction"].fillna("").apply(lambda s: textwrap.fill(str(s), width=80))

In [19]:
from src.agent.generate import render_messages_df

RETURN_COL = ["filename", "avg_mcc_difficulty_bucket",  "scenario", "split"]

messages_df = render_messages_df(
    explained_df,
    df_return_col = RETURN_COL,
    system_tmpl = "masked_system_message.j2",
    user_tmpl = "masked_user_message.j2",
    assistant_tmpl = "masked_assistant_message.j2"
)

messages_df.head()

,filename,avg_mcc_difficulty_bucket,scenario,split,messages
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,1,MASKED_EXPLAIN_CODE,train,"[{'role': 'system', 'content': 'You are a Cryp..."
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,1,MASKED_EXPLAIN_CODE,test,"[{'role': 'system', 'content': 'You are a Cryp..."
2,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,1,MASKED_EXPLAIN_CODE,train,"[{'role': 'system', 'content': 'You are a Cryp..."
3,AES-GCM-SIV-proof/proof/cryptol-specs/TBox.cry,4,MASKED_EXPLAIN_CODE,train,"[{'role': 'system', 'content': 'You are a Cryp..."
4,AES-GCM-SIV-proof/proof/cryptol-specs/Common/A...,4,MASKED_EXPLAIN_CODE,train,"[{'role': 'system', 'content': 'You are a Cryp..."


In [20]:
messages_df["split"].value_counts(dropna=False)

split
train    617
test      61
val       52
Name: count, dtype: int64

In [21]:


train_df = messages_df[messages_df["split"] == "train"].drop(columns=["split"])
test_df = messages_df[messages_df["split"] == "test"].drop(columns=["split"])
val_df = messages_df[messages_df["split"] == "val"].drop(columns=["split"])

train_df.head()

,filename,avg_mcc_difficulty_bucket,scenario,messages
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,1,MASKED_EXPLAIN_CODE,"[{'role': 'system', 'content': 'You are a Cryp..."
2,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,1,MASKED_EXPLAIN_CODE,"[{'role': 'system', 'content': 'You are a Cryp..."
3,AES-GCM-SIV-proof/proof/cryptol-specs/TBox.cry,4,MASKED_EXPLAIN_CODE,"[{'role': 'system', 'content': 'You are a Cryp..."
4,AES-GCM-SIV-proof/proof/cryptol-specs/Common/A...,4,MASKED_EXPLAIN_CODE,"[{'role': 'system', 'content': 'You are a Cryp..."
5,AES-GCM-SIV-proof/proof/asm/cryptol/Asm128.cry,3,MASKED_EXPLAIN_CODE,"[{'role': 'system', 'content': 'You are a Cryp..."


In [22]:
from src.data_s.util import jsonl_append_dedup

jsonl_append_dedup(
    train_df,
    "./data/datasets/train.jsonl",
    overwrite_existing=False
)

jsonl_append_dedup(
    test_df,
    "./data/datasets/test.jsonl",
    overwrite_existing=False
)

jsonl_append_dedup(
    val_df,
    "./data/datasets/validation.jsonl",
    overwrite_existing=False
)

PosixPath('data/datasets/validation.jsonl')